In [ ]:
!pip install pandas dask psutil kaggle --quiet


In [ ]:
from google.colab import files
import os

uploaded = files.upload()

os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)

for filename in uploaded.keys():
    os.rename(filename, os.path.expanduser("~/.kaggle/kaggle.json"))

os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

Saving kaggle.json to kaggle.json


In [ ]:
import os, zipfile


!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store


with zipfile.ZipFile("ecommerce-behavior-data-from-multi-category-store.zip", "r") as z:
    z.extractall(".")

!ls -lh
print("الملفات الموجودة:", os.listdir("."))

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
100% 4.28G/4.29G [00:45<00:00, 179MB/s]
100% 4.29G/4.29G [00:45<00:00, 101MB/s]
total 18G
-rw-r--r-- 1 root root 8.4G Oct 15 13:30 2019-Nov.csv
-rw-r--r-- 1 root root 5.3G Oct 15 13:31 2019-Oct.csv
-rw-r--r-- 1 root root 4.3G Dec  9  2019 ecommerce-behavior-data-from-multi-category-store.zip
drwxr-xr-x 1 root root 4.0K Oct 13 13:43 sample_data
الملفات الموجودة: ['.config', '2019-Oct.csv', '2019-Nov.csv', 'ecommerce-behavior-data-from-multi-category-store.zip', 'sample_data']


In [ ]:
import pandas as pd

csv_file = "/content/2019-Nov.csv"

sample_df = pd.read_csv(csv_file, nrows=5)
print("معاينة أولية للبيانات:")
print(sample_df.head())

معاينة أولية للبيانات:
                event_time event_type  product_id          category_id  \
0  2019-11-01 00:00:00 UTC       view     1003461  2053013555631882655   
1  2019-11-01 00:00:00 UTC       view     5000088  2053013566100866035   
2  2019-11-01 00:00:01 UTC       view    17302664  2053013553853497655   
3  2019-11-01 00:00:01 UTC       view     3601530  2053013563810775923   
4  2019-11-01 00:00:01 UTC       view     1004775  2053013555631882655   

               category_code   brand   price    user_id  \
0     electronics.smartphone  xiaomi  489.07  520088904   
1  appliances.sewing_machine  janome  293.65  530496790   
2                        NaN   creed   28.31  561587266   
3  appliances.kitchen.washer      lg  712.87  518085591   
4     electronics.smartphone  xiaomi  183.27  558856683   

                           user_session  
0  4d3b30da-a5e4-49df-b1a8-ba5943f1dd33  
1  8e5f4f83-366c-4f70-860e-ca7417414283  
2  755422e7-9040-477b-9bd2-6a6e8fd97387  
3  3bfb58

In [ ]:
import time
import psutil
import pandas as pd
import os

start_time = time.time()

chunk_size = 100000
chunks = []
process = psutil.Process(os.getpid())


for i, chunk in enumerate(pd.read_csv(csv_file, chunksize=chunk_size)):
    chunks.append(len(chunk))
    print(f" دفعة {i+1}: {len(chunk)} صفوف ، الذاكرة الحالية: {process.memory_info().rss / 1024 ** 2:.2f} MB")

total_rows = sum(chunks)
elapsed_chunks = time.time() - start_time
memory_chunks = process.memory_info().rss / 1024 ** 2


print(f"عدد الصفوف الكلي: {total_rows}")
print(f"الوقت المستغرق: {elapsed_chunks:.2f} ثانية باستخدام chunksize")

 دفعة 1: 100000 صفوف ، الذاكرة الحالية: 185.51 MB
 دفعة 2: 100000 صفوف ، الذاكرة الحالية: 197.04 MB
 دفعة 3: 100000 صفوف ، الذاكرة الحالية: 201.36 MB
 دفعة 4: 100000 صفوف ، الذاكرة الحالية: 201.36 MB
 دفعة 5: 100000 صفوف ، الذاكرة الحالية: 203.88 MB
 دفعة 6: 100000 صفوف ، الذاكرة الحالية: 196.39 MB
 دفعة 7: 100000 صفوف ، الذاكرة الحالية: 203.89 MB
 دفعة 8: 100000 صفوف ، الذاكرة الحالية: 196.39 MB
 دفعة 9: 100000 صفوف ، الذاكرة الحالية: 200.09 MB
 دفعة 10: 100000 صفوف ، الذاكرة الحالية: 200.09 MB
 دفعة 11: 100000 صفوف ، الذاكرة الحالية: 203.71 MB
 دفعة 12: 100000 صفوف ، الذاكرة الحالية: 196.00 MB
 دفعة 13: 100000 صفوف ، الذاكرة الحالية: 203.50 MB
 دفعة 14: 100000 صفوف ، الذاكرة الحالية: 203.50 MB
 دفعة 15: 100000 صفوف ، الذاكرة الحالية: 203.50 MB
 دفعة 16: 100000 صفوف ، الذاكرة الحالية: 192.18 MB
 دفعة 17: 100000 صفوف ، الذاكرة الحالية: 201.21 MB
 دفعة 18: 100000 صفوف ، الذاكرة الحالية: 201.21 MB
 دفعة 19: 100000 صفوف ، الذاكرة الحالية: 201.21 MB
 دفعة 20: 100000 صفوف ، الذاكرة الحالية:

In [ ]:
!pip install dask

import dask.dataframe as dd
import time
import psutil
import os

start_time = time.time()

df_dask = dd.read_csv(csv_file)

row_count = df_dask.shape[0].compute()

elapsed_dask = time.time() - start_time

process = psutil.Process(os.getpid())
memory_dask = process.memory_info().rss / 1024 ** 2


print(f"عدد الصفوف: {row_count}")
print(f"الوقت المستغرق بـ Dask: {elapsed_dask:.2f} ثانية")
print(f"الذاكرة الحالية بـ Dask: {memory_dask:.2f} MB")

عدد الصفوف: 67501979
الوقت المستغرق بـ Dask: 218.50 ثانية
الذاكرة الحالية بـ Dask: 518.45 MB


In [ ]:
import gzip
import shutil
import time

start_compress = time.time()
with open(csv_file, 'rb') as f_in:
    with gzip.open('data_compressed.csv.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
compress_time = time.time() - start_compress

start_read = time.time()
df_compressed = pd.read_csv('data_compressed.csv.gz', compression='gzip', nrows=50000)
read_time = time.time() - start_read
memory_compressed = process.memory_info().rss / 1024 ** 2

print(f" وقت الضغط: {compress_time:.2f} ثانية")
print(f" وقت القراءة بعد الضغط: {read_time:.2f} ثانية")
print(f"الذاكرة:: {memory_compressed:.2f}  MB")
print(f" حجم الملف قبل وبعد الضغط:")
!ls -lh *.csv*


 وقت الضغط: 710.34 ثانية
 وقت القراءة بعد الضغط: 0.39 ثانية
الذاكرة:: 526.47  MB
 حجم الملف قبل وبعد الضغط:
-rw-r--r-- 1 root root 8.4G Oct 15 13:30 2019-Nov.csv
-rw-r--r-- 1 root root 5.3G Oct 15 13:31 2019-Oct.csv
-rw-r--r-- 1 root root 2.7G Oct 15 13:52 data_compressed.csv.gz


In [ ]:
import pandas as pd
import psutil
import os

memory_compressed = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2


results = pd.DataFrame({
    'الطريقة': ['Pandas + chunksize', 'Dask', 'Pandas + Compression'],
    'الزمن التقريبي (ثواني)': [elapsed_chunks, elapsed_dask, compress_time],
    'الذاكرة (MB)': [memory_chunks, memory_dask, memory_compressed],

})

results

,الطريقة,الزمن التقريبي (ثواني),الذاكرة (MB)
0,Pandas + chunksize,199.994145,145.480469
1,Dask,218.495650,518.449219
2,Pandas + Compression,710.343162,529.402344
